In [150]:
import os
import sys
import random
import gzip
import os
import numpy as np

## paths to files. Do not change this
simInputFile = "Q1/word-similarity-dataset"
#analogyInputFile = "Q1/word-analogy-dataset"
analogyInputFile = "analogy_small.txt"
vectorgzipFile = "Q1/glove.6B.300d.txt.gz"
#vectorTxtFile = "Q1/glove.6B.300d.txt"   # If you extract and use the gz file, use this.
analogyTrainPath = "Q1/wordRep/"
simOutputFile = "Q1/simOutput.csv"
simSummaryFile = "Q1/simSummary.csv"
anaSOln = "Q1/analogySolution.csv"
Q4List = "Q4/wordList.csv"

random.seed(100)

In [151]:
# Similarity Dataset
simDataset = [item.split(" | ") for item in open(simInputFile).read().splitlines()]
# Analogy dataset
analogyDataset = [[stuff.strip() for stuff in item.strip('\n').split('\n')] for item in open(analogyInputFile).read().split('\n\n')]

def vectorExtract(simD = simDataset, anaD = analogyDataset, vect = vectorgzipFile):
    simList = [stuff for item in simD for stuff in item]
    analogyList = [thing for item in anaD for stuff in item[0:4] for thing in stuff.split()]
    simList.extend(analogyList)
    wordList = set(simList)
    print(len(wordList))
    wordDict = dict()
    
    vectorFile = gzip.open(vect, 'r')
    for line in vectorFile:
        if line.split()[0].strip().decode("utf-8") in wordList:
            wordDict[line.split()[0].strip().decode('utf-8')] = line.split()[1:]
    
    
    vectorFile.close()
    print('retrieved', len(wordDict.keys()))
    return wordDict

# Extracting Vectors from Analogy and Similarity Dataset
validateVectors = vectorExtract()

18195
retrieved 16617


In [160]:
import random
import os
import sys

path = "./Q1/wordRep/pairs_from_wordnet/"

#Check if all the words exist in word vector
def vectorexists(wordlist, worddict):
    wordlist = wordlist.split('\t')
    for w in wordlist:
        if(w not in worddict):
            return False
    return True

def writeanalogyfile(path, wordDict):
    filelist = os.listdir(path)
    #print(filelist)
    fout = open('analogy_dataset.txt', 'w')
    for file in filelist:
        alist = []
        qfile = open(path+file, 'r')
        qlist = [line.strip() for line in qfile]
        for otherfiles in filelist:
            if(file != otherfiles):
                f = open(path+otherfiles, 'r')
                l = [line.strip() for line in f]
                #print(file, otherfiles, len(l))
                alist.extend(l)
        for q in qlist:
            if(not vectorexists(q, wordDict)):
                continue
            output = [None]*7
            output[0] = q
            option = random.randint(0,4)
            optionpos = random.randint(0, len(qlist)-1)
            while(not vectorexists(qlist[optionpos], wordDict)):
                optionpos = random.randint(0, len(qlist)-1)
            output[option+1] = qlist[optionpos]
            output[6] = chr(ord('a')+option)
            for i in range(1, 6):
                randop = random.randint(0, len(alist)-1)
                while(not vectorexists(alist[randop], wordDict)):
                    randop = random.randint(0, len(alist)-1)
                if(i!=option+1):
                    output[i] = alist[randop]
            for pair in output:
                pair = pair.replace('\t', ' ')
                fout.write(pair+'\n')
            fout.write('\n\n')
            fout.flush()

#writeanalogyfile(path, wordDict)

In [188]:
def get_analogy_input(anaF, wordDict):
    fin = open(anaF, 'r')
    line = fin.readline()
    trainx = []
    trainy = []
    #trainx = np.zeros((1, 3600))
    #trainy = np.zeros((1,1))
    while(line != ""):
        if(line == '\n'):
            line = fin.readline()
            continue
        #x = np.zeros((1, 300))
        #y = np.zeros((1,1))
        x = []
        y = [0]*5
        for i in range(0, 6):
            line = line.strip().split()
            for w in line:
                if w in wordDict:
                    x.extend(wordDict[w])
                    #print(len(wordDict[w]))
            line = fin.readline()
        answer = line.strip()
        answer = ord(answer)-ord('a')
        y[answer] = 1
        #x = np.array(x)
        if(len(x)<3600):
            line = fin.readline()
            continue
        trainx.append(x)
        trainy.append(y)
        #trainx = np.vstack((trainx, np.array(x).T))
        #trainy = np.vstack((trainy, np.array(y)))
        line = fin.readline()
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    return trainx[1:], trainx[1:], trainy[1:], trainy[1:]

In [177]:
#wordDict = vectorExtract()
trainx, trainy, testx, testy = get_analogy_input(analogyInputFile, wordDict)
print(trainx.shape, trainy.shape)
#print(len(trainx), len(trainy))

(6991, 3600) (6991, 5)


In [172]:
np.min(trainy)

0

In [191]:
# Implementation of a simple MLP network with one hidden layer. Tested on the iris data set.
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

# NOTE: In order to make the code simple, we rewrite x * W_1 + b_1 = x' * W_1'
# where x' = [x | 1] and W_1' is the matrix W_1 appended with a new row with elements b_1's.
# Similarly, for h * W_2 + b_2
import tensorflow as tf
import numpy as np
#from sklearn import datasets
#from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_iris_data():
    """ Read the iris data set and split them into training and test sets """
    iris   = datasets.load_iris()
    data   = iris["data"]
    target = iris["target"]

    # Prepend the column of 1s for bias
    N, M  = data.shape
    all_X = np.ones((N, M + 1))
    all_X[:, 1:] = data

    # Convert into one-hot vectors
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels)[target]  # One liner trick!
    return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)

def trainnetwork(anaF, wordDict):
    #train_X, test_X, train_y, test_y = get_iris_data()
    train_X, test_X, train_y, test_y = get_analogy_input(anaF, wordDict)

    # Layer's sizes
    x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 256                # Number of hidden nodes
    y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(15):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()



In [192]:
trainnetwork(analogyInputFile, wordDict)

Epoch = 1, train accuracy = 27.75%, test accuracy = 27.75%
Epoch = 2, train accuracy = 38.94%, test accuracy = 38.94%
Epoch = 3, train accuracy = 52.72%, test accuracy = 52.72%
Epoch = 4, train accuracy = 67.37%, test accuracy = 67.37%
Epoch = 5, train accuracy = 81.18%, test accuracy = 81.18%
Epoch = 6, train accuracy = 91.90%, test accuracy = 91.90%
Epoch = 7, train accuracy = 96.82%, test accuracy = 96.82%
Epoch = 8, train accuracy = 99.46%, test accuracy = 99.46%
Epoch = 9, train accuracy = 99.91%, test accuracy = 99.91%
Epoch = 10, train accuracy = 100.00%, test accuracy = 100.00%
Epoch = 11, train accuracy = 100.00%, test accuracy = 100.00%
Epoch = 12, train accuracy = 100.00%, test accuracy = 100.00%
Epoch = 13, train accuracy = 100.00%, test accuracy = 100.00%
Epoch = 14, train accuracy = 100.00%, test accuracy = 100.00%
Epoch = 15, train accuracy = 100.00%, test accuracy = 100.00%
